In [5]:
from requests.auth import AuthBase
import json

key = 'your key'

class KAuth(AuthBase):
    def __init__(self, key):
        # setup any auth-related data here
        self.key = key

    def __call__(self, r):
        # modify and return the request
        r.headers['Ocp-Apim-Subscription-Key'] = self.key
        r.headers['Content-Type'] = 'application/json'
        return r

In [198]:
def get_recipe(recipe_id):
    recipe_url = 'https://kesko.azure-api.net/v1/search/recipes'
    body = {'filters' : {'mainCategory': '4', 'subCategory': '28'}}
    recipes_response = requests.post(recipe_url, json=body, auth=KAuth(key))
    recipes_response_text = recipes_response.text
    recipes_response_json = json.loads(recipes_response_text)
    recipe = recipes_response_json['results'][recipe_id]
    name = recipe['Name']
    instructions = recipe['Instructions']
    ingredients = recipe['Ingredients'][0]['SubSectionIngredients']
    image = recipe['PictureUrls'][0]['Normal']
    return name, ingredients, instructions, image
    
def parse_ingredients(ingredients):
    parsed_ingredients = []
    for ingredient in ingredients:
        parsed_ingredient = {}
        parsed_ingredient['name'] = ingredient[0]['IngredientTypeName']
        parsed_ingredient['amount'] = ingredient[0]['Amount']
        parsed_ingredient['unit'] = ingredient[0]['Unit']
        parsed_ingredient['type'] = ingredient[0]['IngredientType']
        parsed_ingredients.append(parsed_ingredient)
    return parsed_ingredients
    
def get_items_for_item_type(item_type):
    products_url = 'https://kesko.azure-api.net/v1/search/products'
    body = {'filters' : {'ingredientType': item_type}}
    items_response = requests.post(products_url, json=body, auth=KAuth(key))
    items_json = json.loads(items_response.text)
    return items_json['results']

def parse_items(items):
    parsed_items = []
    for item in items:
        parsed_item = {}
        parsed_item['ean'] = item['ean']
        parsed_item['name'] = item['labelName']['english']
        parsed_items.append(parsed_item)
    return parsed_items

def get_stores(zip_code = '00180'):
    store_url = 'https://kesko.azure-api.net/v1/search/stores'
    body = {'filters' : {'postCode': zip_code}}
    stores_response = requests.post(store_url, json=body, auth=KAuth(key))
    stores_json = json.loads(stores_response.text)
    return stores_json['results']

def parse_stores(stores):
    parsed_stores = []
    for store in stores:
        parsed_store = {}
        parsed_store['name'] = store['Name']
        parsed_store['id'] = store['Id']
        parsed_stores.append(parsed_store)
    return parsed_stores

def check_availability(ean, store):
    availability_url = 'https://kesko.azure-api.net/v2/products?ean=' + ean
    availability_response = requests.get(availability_url, auth=KAuth(key))
    availability_json = json.loads(availability_response.text)
    availability_stores = availability_json[0]['stores']
    for a_store in availability_stores:
        if a_store['id'] == store['id']:
            return True
    return False


In [202]:
def get_rich_recipe(zip_code, recipe_id, existing_ingredient_types):
    stores = parse_stores(get_stores(zip_code))
    recipe_name, recipe_ingredients, recipe_instructions, recipe_image = get_recipe(recipe_id)
    parsed_ingredients = parse_ingredients(recipe_ingredients)
    rich_ingredients = []
    for ingredient in parsed_ingredients:
        available = 1
        if not ingredient['type'] in existing_ingredient_types:
            available = 0
            items = parse_items(get_items_for_item_type(ingredient['type']))
            for item in items:
                for store in stores:
                    if check_availability(item['ean'],store):
                        available = 1
        ingredient['availability'] = available
        rich_ingredients.append(ingredient)
    return recipe_name, rich_ingredients, recipe_instructions, recipe_image
            
    

In [205]:
a,b,c,d = get_rich_recipe('00180', 1, [])

In [206]:
print(a)
print(b)
print(c) 
print(d)

Teriyakikana
[{'name': 'Soijakastike', 'amount': '1/2', 'unit': 'dl', 'type': '6953', 'availability': 1}, {'name': 'Riisiviinietikka', 'amount': '1/2', 'unit': 'dl', 'type': '6740', 'availability': 0}, {'name': 'Ruokokidesokeri', 'amount': '2', 'unit': 'rkl', 'type': '6795', 'availability': 1}, {'name': 'Valkosipuli, tuore', 'amount': '1', 'unit': '', 'type': '15809', 'availability': 0}, {'name': 'Inkivääri, jauhettu', 'amount': '1', 'unit': 'tl', 'type': '5566', 'availability': 1}, {'name': 'Maissitärkkelysjauho', 'amount': '1', 'unit': 'tl', 'type': '6035', 'availability': 1}]
# Mittaa kastikkeen ainekset pieneen kattilaan ja sekoita tasaiseksi kierrevatkaimella. Kuumenna koko ajan sekoittaen, kunnes seos paksuuntuu. Anna kastikkeen jäähtyä.
# Pyyhi broilereiden pinta talouspaperilla ja pane ne marinoitumaan teriyakikastikkeeseen huoneenlämpöön puoleksi tunniksi ennen paistamista.
# Paista broilerit kypsiksi öljyssä pannulla. Sivele fileille marinadia paistamisen aikana. Viipaloi kyp